# TODO: 概要

# ライブラリのインポート

In [1]:
import os
import json
import asyncio
import datetime

from dotenv import load_dotenv, find_dotenv

from IPython.display import Image, display

from azure.identity.aio import DefaultAzureCredential
from azure.ai.agents.models import (
    FileInfo, FileSearchTool, VectorStore,
    CodeInterpreterTool, FilePurpose,
    ListSortOrder
)

from semantic_kernel.agents import (
    ChatCompletionAgent, ChatHistoryAgentThread,
    AzureAIAgent, AzureAIAgentSettings, AzureAIAgentThread
)
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStreamableHttpPlugin
from semantic_kernel.contents import (
    ChatMessageContent, FunctionCallContent, FunctionResultContent, AuthorRole, TextContent
)


# 環境変数の取得

In [2]:
load_dotenv(override=True)

PROJECT_ENDPOINT=os.getenv("PROJECT_ENDPOINT")
AZURE_DEPLOYMENT_NAME=os.getenv("AZURE_DEPLOYMENT_NAME")
AZURE_OPENAI_ENDPOINT=os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY=os.getenv("AZURE_OPENAI_API_KEY")

FOUNDRY_CODE_INTERPRETER_AGENT_ID=os.getenv("FOUNDRY_CODE_INTERPRETER_AGENT_ID")

# ユーティリティ関数

In [3]:
async def print_thread_message_details(thread: str):
    """
    スレッドのメッセージ詳細を表示します。

    Args:
        thread (str): スレッドのインスタンス
    """
    async for message in thread.get_messages():
        print("-----")

        for item in message.items:
            if isinstance(item, FunctionCallContent):
                print(f"[Function Calling] by {message.ai_model_id}")
                print(f" - Function Name : {item.name}")
                print(f" - Arguments     : {item.arguments}")

            elif isinstance(item, FunctionResultContent):
                print(f"[Function Result]")
                # 文字列のデコード変換
                if isinstance(item.result, str):
                    try:
                        decoded = json.loads(item.result)
                        print(f" - Result        : {decoded}") # デコード成功時は変換後の値を表示
                    except json.JSONDecodeError:
                        print(f" - Result        : {item.result}")  # デコード失敗時はそのまま
                else:
                    print(f" - Result        : {item.result}")

            elif isinstance(item, TextContent):
                if message.name:
                    print(f"[Agent Response] from {message.ai_model_id}")
                else:
                    print("[User Message]")
                print(f" - Content       : {item.text}")

            else:
                print(f"[Unknown Item Type] ({type(item).__name__})")
                print(f" - Raw Item      : {item}")


def log_with_timestamp(message: str) -> None:
    """
    現在時刻付きでメッセージを標準出力にログとして表示します。

    Args:
        message (str): 出力するログメッセージ。
    """
    timestamp = datetime.datetime.now().strftime("%H:%M:%S.%f")[:-3]
    print(f"[{timestamp}] {message}")

In [4]:
async def agent_run_outputs(thread_id, agents_client, target_dir="./output_images"):
    """
    指定したスレッドIDのRun実行結果（テキスト・画像）をNotebook上に表示＆画像は保存。
    """
    messages = agents_client.messages.list(thread_id=thread_id, order=ListSortOrder.ASCENDING)
    os.makedirs(target_dir, exist_ok=True)

    async for message in messages:
        # テキスト出力
        if message.text_messages:
            for txt in message.text_messages:
                print(f"{message.role.upper()}: {txt.text.value}")

        # 画像出力
        if hasattr(message, "image_contents"):
            for image_content in message.image_contents:
                file_id = image_content.image_file.file_id
                file_name = f"{file_id}_image_file.png"

                await agents_client.files.save(
                    file_id=file_id,
                    file_name=file_name,
                    target_dir=target_dir
                )
                print(f"Saved image: {file_name}")
                display(Image(filename=f"{target_dir}/{file_name}"))


# プラグインの作成

## MCP プラグイン

In [5]:
# MCPのエンドポイントURL
MCP_URL = "http://127.0.0.1:8000/mcp/"


mcp_plugin = MCPStreamableHttpPlugin(
    name="mcp_plugin",
    url=MCP_URL
)

await mcp_plugin.connect()

# クライアントの初期化

In [6]:
# AzureAIAgent クライアントを初期化
project_client = AzureAIAgent.create_client(
    endpoint=PROJECT_ENDPOINT,
    credential=DefaultAzureCredential()
)

In [7]:
# Chat Completion API クライアントの初期化
azure_completion_service  = AzureChatCompletion(
    service_id="azure_completion_agent",
    deployment_name=AZURE_DEPLOYMENT_NAME,
    endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_API_KEY
)

# プラグインエージェントの作成

## ChatCompletionAgent の作成

In [8]:
mcp_agent = ChatCompletionAgent(
    service=azure_completion_service,
    name="mcp_agent",
    instructions=(
        "あなたはECサイトのデータ取得エンジニアです。"
        "PostgreSQLの各種マスター、注文、ユーザー情報と、CosmosDBに格納された商品レビューデータの取得を行います。"
    ),
    plugins=[mcp_plugin],
)

## AzureAIAgent の作成

In [9]:
code_interpreter_agent_definition = await project_client.agents.get_agent(agent_id=FOUNDRY_CODE_INTERPRETER_AGENT_ID)

In [10]:
code_interpreter_agent = AzureAIAgent(
    client=project_client,
    definition=code_interpreter_agent_definition,
)

# 親エージェントの作成

In [11]:
analytics_agent = ChatCompletionAgent(
    service=azure_completion_service,
    name="analytics_agent",
    instructions=(
        "あなたはECサイトの分析エージェントです。以下のプラグインを利用して、データの分析や可視化を行います。"
        "- mcp_plugin: PostgreSQLの各種マスター、注文、ユーザー情報と、CosmosDBに格納されたレビューデータを取得"
        "- code_interpreter_agent: Pythonコードを実行し、データの分析や可視化を実行"
    ),
    plugins=[mcp_plugin, code_interpreter_agent],
)

# スレッドの作成

In [12]:
# Thread の作成
thread = ChatHistoryAgentThread()
print(f"Created Thread. THREAD_ID: {thread.id}")


Created Thread. THREAD_ID: thread_1c91a85e85504a86ae4333619df09bf9


## レスポンスを取得

In [13]:
formatted_json = json.dumps(analytics_agent.model_dump(), indent=2, ensure_ascii=False, default=str)
print(formatted_json)

{
  "arguments": null,
  "description": null,
  "id": "bb47f1b7-5d42-458c-83a4-e8180786ecae",
  "instructions": "あなたはECサイトの分析エージェントです。以下のプラグインを利用して、データの分析や可視化を行います。- mcp_plugin: PostgreSQLの各種マスター、注文、ユーザー情報と、CosmosDBに格納されたレビューデータを取得- code_interpreter_agent: Pythonコードを実行し、データの分析や可視化を実行",
  "kernel": {
    "services": {
      "azure_completion_agent": {
        "ai_model_id": "gpt-4.1",
        "service_id": "azure_completion_agent"
      }
    },
    "ai_service_selector": "<semantic_kernel.services.ai_service_selector.AIServiceSelector object at 0x000001DED22C3B10>",
    "plugins": {
      "mcp_plugin": {
        "name": "mcp_plugin",
        "description": null,
        "functions": {
          "get_all_categories": {
            "metadata": {
              "name": "get_all_categories",
              "plugin_name": "mcp_plugin",
              "description": "\n    全カテゴリ一覧を取得します。\n\n    :return: JSON形式でカテゴリの一覧を返します。\n    :rtype: str\n    ",
              "parameters": [],
              

In [14]:
user_message = (
    "Surface Laptopの2024年月別販売数量をグラフを出力して。"
)

response = await analytics_agent.get_response(
    messages=[user_message],
    thread=thread,
)

print(response)

Surface Laptop（product_id: 2）の2024年の販売数量を集計するため、100件の注文データからSurface Laptopだけをピックアップして数量を月別で集計します。

一部のデータ抽出例（4月分、1〜10まで）：
- 2024-04-15の注文（order_id:1,2,3,4,5,6,7,8,9,10）は全てSurface Laptop（product_id:2）。数量は {6, 4, 4, 6, 4, 5, 5, 5, 5, 6}
- それ以外にも各月で同様に抽出

この処理を全注文データに対して行い、月ごとに合算したグラフを生成します。

続けて全データ100件分を自動抽出し、月別棒グラフを出力します。少々お待ちください。


In [15]:
async for message in thread.get_messages():
    # message.id は存在しない
    print(f"Role: {message.role}")
    if hasattr(message, "name"):
        print(f"Name: {message.name}")
    for item in message.items:
        if isinstance(item, FunctionCallContent):
            print(f"Function Call - Name: {item.name}, Arguments: {item.arguments}")
        elif isinstance(item, FunctionResultContent):
            print(f"Function Result - Result: {item.result}")
        elif isinstance(item, TextContent):
            print(f"Text Content: {item.text}")
        else:
            print(f"Unknown Item Type: {type(item)}")


Role: AuthorRole.USER
Name: None
Text Content: Surface Laptopの2024年月別販売数量をグラフを出力して。
Role: AuthorRole.ASSISTANT
Name: analytics_agent
Function Call - Name: mcp_plugin-get_all_categories, Arguments: {}
Role: AuthorRole.TOOL
Name: analytics_agent
Function Result - Result: [TextContent(inner_content=TextContent(type='text', text='[{"category_id": 1, "category_name": "PC・タブレット"}, {"category_id": 2, "category_name": "ゲーム・エンタメ"}, {"category_id": 3, "category_name": "ソフトウェア"}, {"category_id": 4, "category_name": "アクセサリ"}, {"category_id": 5, "category_name": "サービス"}]', annotations=None, meta=None), ai_model_id=None, metadata={}, content_type='text', text='[{"category_id": 1, "category_name": "PC・タブレット"}, {"category_id": 2, "category_name": "ゲーム・エンタメ"}, {"category_id": 3, "category_name": "ソフトウェア"}, {"category_id": 4, "category_name": "アクセサリ"}, {"category_id": 5, "category_name": "サービス"}]', encoding=None)]
Role: AuthorRole.ASSISTANT
Name: analytics_agent
Function Call - Name: mcp_plugin-get_prod

In [16]:
# thread.get_messages()は非同期イテレータを返すため、async forを使用
async for message in thread.get_messages():
    print(f"Message ID: {message.id}")
    print(f"Role: {message.role}")
    for item in message.items:
        if isinstance(item, FunctionCallContent):
            print(f"Function Call - Name: {item.name}, Arguments: {item.arguments}")
        elif isinstance(item, FunctionResultContent):
            print(f"Function Result - Result: {item.result}")
        elif isinstance(item, TextContent):
            print(f"Text Content: {item.text}")
        else:
            print(f"Unknown Item Type: {type(item)}")
# print(tmp)
# print(dir(tmp))

AttributeError: 'ChatMessageContent' object has no attribute 'id'